In [11]:
import pandas as pd

In [12]:
#importing asset report that you dl from your client's cms
asset = pd.read_csv('asset_export (1).csv')

In [13]:
asset_1 = asset.copy()

In [14]:
asset_1.columns

Index(['asset_id', 'Identifier', 'Asset type', 'episode_title', 'Custom ID',
       'asset_label', 'Time Created'],
      dtype='object')

In [15]:
asset_1 = asset_1.fillna(0)

In [16]:
#asset_2 is for later use during the assets mapping process.
asset_2 = asset_1[['asset_id','episode_title','asset_label']]

In [17]:
#extracting asset label column from the asset report,
#this is the column that contains relevant show/series names that you need for asset mapping.

asset_1_assetlabels = asset_1[['asset_label']]
asset_1_assetlabels['asset_label'] = asset_1_assetlabels['asset_label'].str.lower()

/Users/azhoula/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
asset_1_assetlabels.head()

,asset_label
0,reference
1,reference
2,reference
3,reference
4,reference


In [19]:
#removing any depulicated values
asset_1_assetlabels = asset_1_assetlabels.drop_duplicates()

In [22]:
#spliting any value that has a '|'
new = asset_1_assetlabels['asset_label'].str.split(",", expand = True)
new.head()

,0,1,2
0,reference,None,None
6,oprah winfrey network,None,None
7,ambitions,None,None
9,NaN,NaN,NaN
16,digital original,None,None


In [23]:
asset_1_assetlabels['asset_labels1'] = new[0]
asset_1_assetlabels['asset_labels2'] = new[1]

In [24]:
#In my case, column 1 and 2 are the columns that need to be filter again (remove duplicated values)
new_df1 = asset_1_assetlabels[['asset_labels1']].drop_duplicates()
new_df1.count()

asset_labels1    26
dtype: int64

In [25]:
new_df2 = asset_1_assetlabels[['asset_labels2']].drop_duplicates()
new_df2.head()

,asset_labels2
0,None
38,s1
77,ambitions
148,the oprah winfrey show
203,reference


In [26]:
#combining the filtered/cleaned columns (df1 & df2)
frame = new_df1.append(new_df2, ignore_index=True)
frame = frame.fillna('none')
frame = frame['asset_labels1'].append(frame['asset_labels2'])
frame = pd.DataFrame(frame)
frame.columns = ['show_names']
frame.count()

show_names    64
dtype: int64

In [27]:
#Now, you have a list of show/series names that you will be needed for next step, which is mapping.
frame = frame.drop_duplicates()
frame.head()

,show_names
0,reference
1,oprah winfrey network
2,ambitions
3,none
4,digital original


In [28]:
#print(frame)

In [29]:
#frames.to_csv('frame.csv')

import the claim report and merge it with asset_2 df that was created eariler, based on asset id.

In [30]:
#import claim report
claim = pd.read_csv('claim_report.csv')

/Users/azhoula/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
claim_1 = claim.copy()

In [32]:
#there are so many columns in the claim report, but we only need 2 columns - asset id and video title
claim_1 = claim_1[['asset_id','video_title']]

In [33]:
#merge asset_2 onto claim report
merger = asset_2.merge(claim_1, how = 'left', left_on = 'asset_id', right_on = 'asset_id' )

In [34]:
#you have created a new df that contains a complete list of asset ids that have active claim(s)
#these are the assets that you want to prioritize for your mapping project
merger = merger.fillna(0)
merger.head()

,asset_id,episode_title,asset_label,video_title
0,A453541124396377,EP206 LETTERMAN,Reference,Oprah Log
1,A453541124396377,EP206 LETTERMAN,Reference,LATE SHOW with David Letterman - It Ain't Opra...
2,A453541124396377,EP206 LETTERMAN,Reference,Sexual Harassment Project
3,A453541124396377,EP206 LETTERMAN,Reference,Опра Уинфри новая глава Дэвид Леттерман
4,A453541124396377,EP206 LETTERMAN,Reference,David Letterman on depression


In [35]:
merger.count()

asset_id         1988
episode_title    1988
asset_label      1988
video_title      1988
dtype: int64

In [36]:
#scanning through all 3 columns (video_title, episode_title, and asset_label) against the extracted show names

df = pd.DataFrame()

for show in frame.show_names:
    
    master_df = pd.DataFrame()
    
    mask = merger['episode_title'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask])
    master_df["show_names"] = show
    mask2 = merger['video_title'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask2])
    master_df["show_names"] = show
    mask3 = merger['asset_label'].str.contains(show,case = False,na=False)
    master_df = master_df.append(merger[mask3])
    master_df["show_names"] = show
    
    df = df.append(master_df, ignore_index=True)

In [37]:
df.columns

Index(['asset_id', 'episode_title', 'asset_label', 'video_title',
       'show_names'],
      dtype='object')

In [38]:
df = df[['asset_id','show_names','asset_label','episode_title','video_title']]

In [39]:
df_1 = df.groupby(['asset_id','show_names'])

In [40]:
#df_1.head()
df_1.count()

,,asset_label,episode_title,video_title
asset_id,show_names,,,
--x7RzGYx40,oprah winfrey network,1,1,1
-H51xs7VEcE,oprah winfrey network,1,1,1
-KnPHWckOnc,oprah winfrey network,1,1,1
-QH13xxNQeQ,oprah winfrey network,1,1,1
-ajFelpKD7g,oprah winfrey network,1,1,1
...,...,...,...,...
TEIiDAaRCbI,reference,3,3,3
TsVo56Z8_Bc,the judds,2,2,2
Xase0vINA5o,oprah winfrey network,1,1,1


In [41]:
df_1.count().reset_index().to_csv('new2.csv')